In [1]:
import pandas as pd
import os
# import textstat
import re
import json
import requests
import string
from datetime import datetime
import numpy as np
from tqdm import tqdm

import uszipcode
from uszipcode import SearchEngine

from textblob import TextBlob

# import nltk
# nltk.download('all')
# from nltk.tokenize import sent_tokenize
# from nltk.corpus import stopwords
# from nltk import word_tokenize, pos_tag
# from nltk.stem import WordNetLemmatizer

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import NMF
# from sklearn.feature_extraction import text

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# 1，add review-level features (text mining)

In [9]:
data = pd.read_csv('../Data/reviews.csv')

/var/folders/b1/83m83_q102d2q6br4ttyzw8m0000gn/T/ipykernel_66261/3833517184.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/reviews.csv')


In [68]:
data = pd.read_csv('../Data/reviews.csv', index_col=0)

# calculate fog index
data['Fog_index'] = data['contents'].apply(lambda x: textstat.gunning_fog(x))

# calculate review length (number of words)
word_cnt = data["contents"].apply(lambda x: len(x.split(" ")))
data["Sent_length"] = word_cnt

# calculate average word length
avg_word_len = data["contents"].apply(lambda x: np.mean([len(w) for w in x.split(" ")]))
data["Avg_word_length"] = avg_word_len

# calculate subjectivity
def get_subjectivity(text: str)->float:
    return TextBlob(text).sentiment.subjectivity
data["Subjectivity"] = data["contents"].apply(get_subjectivity)

def get_polarity(text: str)->float:
    return TextBlob(text).sentiment.polarity
data["Polarity"] = data["contents"].apply(get_polarity)

data['timestamps']  = pd.to_datetime(data['dates'])

data.to_csv('../Data/reviews.csv', index=False)

/var/folders/b1/83m83_q102d2q6br4ttyzw8m0000gn/T/ipykernel_52420/2033343111.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/reviews.csv', index_col=0)


In [18]:
data.to_csv('../Data/reviews.csv', index=False)

In [21]:
data['Name']

0          Grand Lux Cafe
1          Grand Lux Cafe
2          Grand Lux Cafe
3          Grand Lux Cafe
4          Grand Lux Cafe
                ...      
1679894        Yard House
1679895        Yard House
1679896        Yard House
1679897        Yard House
1679898        Yard House
Name: Name, Length: 1679899, dtype: object

In [17]:
data.columns = ['_'.join(col.split(' ')) for col in data.columns]

# 2, Use NMF to extract topics from reviews

In [4]:
data = pd.read_csv('../Data/reviews.csv')

/Users/aoqu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, 
    remove punctuation, remove read errors,
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('�', ' ', text)
    return text

data["contents"] = data["contents"].apply(lambda x: clean_text_round1(x))
data.head()

,Name,Phone,Lat,Long,On_Yelp,Treated_Yelp,Review_count,Link,ID,Categories,...,funny,cool,elite,nphotos_review,checkin,complain_line,complain_service,Fog_index,Sent_length,Avg_word_length
0,Grand Lux Cafe,1.713626e+10,29.740093,-95.472809,Y,N,880.0,https://www.yelp.com/biz/grand-lux-cafe-housto...,GK8YWuqjNkVaW1x-D4PdnA,"['Desserts', 'American (Traditional)', 'Seafood']",...,0.0,0.0,Y,0.0,0.0,0,0,6.12,61,3.885246
1,Grand Lux Cafe,1.713626e+10,29.740093,-95.472809,Y,N,880.0,https://www.yelp.com/biz/grand-lux-cafe-housto...,GK8YWuqjNkVaW1x-D4PdnA,"['Desserts', 'American (Traditional)', 'Seafood']",...,0.0,0.0,N,0.0,0.0,0,1,5.55,65,3.923077
2,Grand Lux Cafe,1.713626e+10,29.740093,-95.472809,Y,N,880.0,https://www.yelp.com/biz/grand-lux-cafe-housto...,GK8YWuqjNkVaW1x-D4PdnA,"['Desserts', 'American (Traditional)', 'Seafood']",...,3.0,3.0,Y,4.0,4.0,0,0,6.17,243,4.251029
3,Grand Lux Cafe,1.713626e+10,29.740093,-95.472809,Y,N,880.0,https://www.yelp.com/biz/grand-lux-cafe-housto...,GK8YWuqjNkVaW1x-D4PdnA,"['Desserts', 'American (Traditional)', 'Seafood']",...,0.0,0.0,Y,0.0,0.0,0,0,6.20,39,4.256410
4,Grand Lux Cafe,1.713626e+10,29.740093,-95.472809,Y,N,880.0,https://www.yelp.com/biz/grand-lux-cafe-housto...,GK8YWuqjNkVaW1x-D4PdnA,"['Desserts', 'American (Traditional)', 'Seafood']",...,0.0,0.0,Y,1.0,2.0,0,0,4.63,19,5.947368


In [9]:
# Add additional stop words since we are recreating the document-term matrix
stop_words_noun_agg = text.ENGLISH_STOP_WORDS
# Create a document-term matrix with only nouns
# Store TF-IDF Vectorizer
tv_noun = TfidfVectorizer(stop_words=stop_words_noun_agg, ngram_range = (1,1), max_df = .8, min_df = .01)
# Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
data_tv_noun = tv_noun.fit_transform(data['contents'])
# Create data-frame of Doc-Term Matrix with nouns as column names
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names())
# Set President's Names as Index
data_dtm_noun.index = data.index
# Visually inspect Document Term Matrix
data_dtm_noun.head()

,able,absolutely,accommodating,actually,add,added,afternoon,ago,ahead,alfredo,...,write,wrong,yard,year,years,yelp,yes,yesterday,young,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.100552,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [10]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    '''Given an NMF model, feature_names, and number of top words, print topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [11]:
nmf_model = NMF(5, max_iter=100)
# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
doc_topic = nmf_model.fit_transform(data_dtm_noun)
# Extract top words from the topic-term matrix 'H' 
display_topics(nmf_model, tv_noun.get_feature_names(), 5)

/Users/aoqu/opt/anaconda3/envs/rosetta/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/Users/aoqu/opt/anaconda3/envs/rosetta/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"



Topic  0
minutes, table, order, came, food

Topic  1
great, service, food, amazing, awesome

Topic  2
good, food, service, really, pretty

Topic  3
chicken, salad, ordered, steak, cheese

Topic  4
place, wings, bar, love, happy


In [15]:
display_topics(nmf_model, tv_noun.get_feature_names(), 20)


Topic  0
minutes, table, order, came, food, manager, time, took, asked, server, wait, got, said, service, waitress, didn, told, waited, just, seated

Topic  1
great, service, food, amazing, awesome, server, friendly, excellent, definitely, delicious, best, experience, atmosphere, staff, love, attentive, recommend, time, thank, wonderful

Topic  2
good, food, service, really, pretty, nice, bad, slow, restaurant, chain, prices, better, atmosphere, time, times, price, little, quality, place, friendly

Topic  3
chicken, salad, ordered, steak, cheese, pizza, like, shrimp, sauce, burger, delicious, got, meal, fries, just, bread, menu, pasta, amp, cooked

Topic  4
place, wings, bar, love, happy, hour, beer, like, just, don, ve, area, game, menu, come, games, best, drinks, staff, watch


In [65]:
nmf_topics = {0: "Wait times", 1: "Quality overall", 2: "Responsiveness", 3: "Food quality", 4: "Atmosphere"}
for index, topic in nmf_topics.items():
    data[topic] = doc_topic[:, index]

In [7]:
data.to_csv('../Data/reviews.csv', index=False)

# 3, Process restaurant data and Compute Amenity Score

In [33]:
restaurants = pd.read_csv('../Data/All_restaurants_finalized.csv', index_col=0)

for index, row in restaurants.drop_duplicates(subset=['Name']).iterrows():

    name, restaurant_id = row['Name'], row['ID']
        
    url = f"https://api.yelp.com/v3/businesses/{restaurant_id}"
    api_key =  "xP0c_9V4xs13EnNoDZFnSFf_dqrFYL9Fqs9ClXm6C9TZHaMMidD9SGVupd0v5NaMSjnpiteshQK539GESDLs25NKNExV6ua9iy-50ntIq3xqcbbkc2GTCNnm4IyqXnYx"

    headers = {
        "accept": "application/json",
        "Authorization": 'Bearer %s' % api_key
    }

    response = requests.get(url, headers=headers)

    data = json.loads(response.text)
    restaurants.loc[restaurants['Name'] == name, 'Price'] = data['price']

restaurants = restaurants[restaurants['Price'] == '$$']

In [35]:
restaurants.reset_index(inplace=True, drop=True)

In [37]:
data = pd.read_csv('../Data/reviews.csv')
restaurants = restaurants.merge(data[['ID', 'Amenities']].drop_duplicates(subset='ID'), on='ID', how='left')
restaurants = restaurants.dropna(subset=['Amenities'])
restaurants = restaurants.reset_index().drop('index', axis=1)

/var/folders/b1/83m83_q102d2q6br4ttyzw8m0000gn/T/ipykernel_38782/259220093.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/reviews.csv')


In [38]:
# Process amenity information
amenity_dict = {'Wi-Fi': ['Free'],
                'Has TV': ['Yes'],
                'Good for Kids': ['Yes'],
                'Good for Groups': ['Yes'],
                'Has Pool Table': ['Yes'],
                }
parking_dict = ['Private Lot', 'Garage', 'Private Lot', 'Street', 'Valet', 'Validated', 'Yes']

def get_amenity_score(amenities: str) -> int:
    amenities = {x.split(':')[0]: x.split(':')[1] for x in amenities.split(',') if ':' in x}
    score = 0
    for key, value in amenity_dict.items():
        if key in amenities:
            if any(x in amenities[key] for x in value):
                score += 1
    return score

def check_parking(x: str) -> int:
    pattern = r",Parking.*?,"
    info = re.findall(pattern, x)
    if len(info) > 0:
        return int(info[0].split(':')[1][:-1] in parking_dict)
    return 0

restaurants['Amenity_score'] = restaurants['Amenities'].apply(get_amenity_score)
restaurants['Takes_reservations'] = restaurants['Amenities'].apply(lambda x: 1 if 'Takes Reservations:Yes' in x else 0)
restaurants['Offers_delivery'] = restaurants['Amenities'].apply(lambda x: 1 if 'Delivery:Yes' in x else 0)
restaurants['Parking_available'] = restaurants['Amenities'].apply(lambda x: check_parking(x))

In [39]:
start_date = datetime(2015, 2, 28)
end_date = datetime(2019, 2, 28)
treatment_date = datetime(2017, 2, 28)

data['dates'] = pd.to_datetime(data['dates'])
data = data[(data['dates'] >= start_date) & (data['dates'] <= end_date)]

In [40]:
# assign unique Geo_Code to each city

zipcode = data['Zip'][2]
search = SearchEngine()

def get_state(zipcode: str) -> str:
    try:
        return search.by_zipcode(int(zipcode)).state
    except:
        return None

restaurants['State'] = restaurants['Zip'].apply(get_state)
restaurants['Geo_code'] = restaurants['City'] + '_' + restaurants['State']
restaurants.dropna(subset=['Geo_code'], inplace=True)
restaurants = restaurants.reset_index().drop('index', axis=1)
restaurants['City_code'] = restaurants['Geo_code'].astype('category').cat.codes

In [41]:
# calculate review count and average rating before and after treatment date for PSM
for index, row in tqdm(restaurants.iterrows()):
    review_before = data[(data['ID'] == row['ID']) & (data['dates'] < treatment_date)]
    review_after = data[(data['ID'] == row['ID']) & (data['dates'] >= treatment_date)]
    review_count_before = review_before.shape[0]
    review_count_after = review_after.shape[0]
    review_rating_before = review_before['ratings'].mean()
    review_rating_after = review_after['ratings'].mean()
    restaurants.loc[index, 'Review_count_before'] = review_count_before
    restaurants.loc[index, 'Review_count_after'] = review_count_after
    restaurants.loc[index, 'Rating_before'] = review_rating_before
    restaurants.loc[index, 'Rating_after'] = review_rating_after

# make sure the sample size is large enough for before and after
restaurants = restaurants[(restaurants['Review_count_before'] > 30) & (restaurants['Review_count_after'] > 30)]

6618it [09:31, 11.59it/s]


In [47]:
restaurants = restaurants[restaurants['Treated_NonYelp'] == 'N']

In [57]:
restaurants.reset_index(inplace=True, drop=True)

In [58]:
psm_data = pd.DataFrame()
all_cats = set([cat for categories in restaurants['Categories'] for cat in categories.split(', ')])
for index, cat in enumerate(all_cats):
    tmp = restaurants[restaurants['Categories'].apply(lambda x: True if cat in x else False)]
    tmp['Match'] = index
    psm_data = psm_data.append(tmp)

psm_data.to_csv("../Data/psm_data.csv", index=False)

/var/folders/b1/83m83_q102d2q6br4ttyzw8m0000gn/T/ipykernel_38782/3482233810.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Match'] = index
/var/folders/b1/83m83_q102d2q6br4ttyzw8m0000gn/T/ipykernel_38782/3482233810.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  psm_data = psm_data.append(tmp)
/var/folders/b1/83m83_q102d2q6br4ttyzw8m0000gn/T/ipykernel_38782/3482233810.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [ ]:
restaurants.to_csv("../Data/restaurants(processed).csv", index=False)

# 4, Process SafeGraph data 

In [37]:
safegraph1 = pd.read_csv('../Data/SafeGraphRaw/safegraph_data_purchase_jul-20-2022/your_data_jul_20_2022_1112pm.csv')
safegraph2 = pd.read_csv('../Data/SafeGraphRaw/safegraph_data_purchase_jul-20-2022 (1)/your_data_jul_20_2022_0915pm.csv')
safegraph = pd.concat([safegraph1, safegraph2], axis=0)
safegraph = safegraph.reset_index().drop(columns=['index'])

In [42]:
safegraph.columns

Index(['placekey', 'parent_placekey', 'location_name', 'street_address',
       'city', 'region', 'postal_code', 'safegraph_brand_ids', 'brands',
       'date_range_start', 'date_range_end', 'raw_visit_counts',
       'raw_visitor_counts', 'visits_by_day', 'poi_cbg', 'visitor_home_cbgs',
       'visitor_daytime_cbgs', 'visitor_country_of_origin',
       'distance_from_home', 'median_dwell', 'bucketed_dwell_times',
       'related_same_day_brand', 'related_same_month_brand',
       'popularity_by_hour', 'popularity_by_day', 'device_type',
       'normalized_visits_by_state_scaling',
       'normalized_visits_by_total_visits',
       'normalized_visits_by_total_visitors',
       'normalized_visits_by_region_naics_visits',
       'normalized_visits_by_region_naics_visitors', 'top_category',
       'sub_category', 'naics_code', 'latitude', 'longitude',
       'iso_country_code', 'phone_number', 'open_hours', 'category_tags',
       'opened_on', 'closed_on', 'tracking_closed_since', 'geomet